In [1]:
%%file wpa.py

from numpy import *

def limiter(r,lim_choice='MC'):
    wlimiter = empty(r.shape)
    if lim_choice == None:
        wlimiter = ones(r.shape)
    elif lim_choice == 'minmod':        
        # wlimitr = dmax1(0.d0, dmin1(1.d0, r))
        wlimiter = maximum(0,minimum(1,r))
    elif lim_choice == 'superbee':
        # wlimitr = dmax1(0.d0, dmin1(1.d0, 2.d0*r), dmin1(2.d0, r))
        a1 = minimum(1,2*r)
        a2 = minimum(2,r)        
        wlimiter = maximum(0,maximum(a1,a2))
    if lim_choice == 'MC':
        # c = (1.d0 + r)/2.d0
        # wlimitr = dmax1(0.d0, dmin1(c, 2.d0, 2.d0*r))
        c = (1 + r)/2
        wlimiter = maximum(0,minimum(c,minimum(2,2*r)))
    elif lim_choice == 'vanleer':
        # wlimitr = (r + dabs(r)) / (1.d0 + dabs(r))
        wlimiter = (r + abs(r))/(1 + abs(r))
            
    return wlimiter


# ----------------------------------------------------
# Wave propagation algorithm
# 
# Adapted from the Clawpack package (www.clawpack.org)
# -----------------------------------------------------

def claw1(ax, bx, N, Tfinal, M, \
          rp=None, \
          qinit=None, \
          bc=None, \
          limiter_choice='MC', \
          second_order=True):

    dx = (bx-ax)/N
    xe = linspace(ax,bx,N+1)  # Edge locations
    xc = xe[:-1] + dx/2       # Cell-center locations


    tvec = linspace(0,Tfinal,M+1)
    dt = Tfinal/M
    
    assert rp is not None,    'No user supplied Riemann solver'
    assert qinit is not None, 'No user supplied initialization routine'
    assert bc is not None,    'No user supplied boundary conditions'

    
    t = 0
    q0 = qinit(xc)
    
    Q = zeros((N,M+1))
    Q[:,0] = q0.copy()

    q = q0.copy()
    dtdx = dt/dx
    for n in range(0,M):
        t = tvec[n]

        # Add 2 ghost cells at each end of the domain;  
        q_ext = bc(q)

        # Get waves, speeds and fluctuations
        waves, s, amdq, apdq = rp(q_ext)
        
#         if n == 0:
#             print("Waves : {:d} entries".format(waves.shape[0]))
#             print("apdq  : {:d} entries".format(apdq.shape[0]))
        
        # First order update
        q = q - dtdx*(apdq[1:-2] + amdq[2:-1])

        # Second order corrections (with possible limiter)
        if second_order:    
            s = s[1:-1]  # Remove unneeded ghost cell values added by Riemann solver.
            if limiter_choice is not None:
                wl = waves[:-2]*waves[1:-1]  # At interfaces 0 to N+1
                wr = waves[2:]*waves[1:-1]
                w2 = waves[1:-1]*waves[1:-1]
                
                # Create mask to avoid dividing by 0. 
                m = w2 > 0
                r = ones(w2.shape)

                r[m] = where(s[m] > 0,  wl[m]/w2[m],1)
                r[m] = where(s[m] <= 0, wr[m]/w2[m],1)
                
                # r[~m] = 1

                wlimiter = limiter(r,limiter_choice)                
            else:
                wlimiter = 1
                
            waves = waves[1:-1]*wlimiter                
            cxx = 0.5*abs(s)*(1 - abs(s)*dtdx)*waves
            Fp = cxx  # Second order derivatives
            Fm = cxx
        
            # update with second order corrections
            q -= dtdx*(Fp[1:] - Fm[:-1])
        
        Q[:,n+1] = q
        
    return Q, xc, tvec



Overwriting wpa.py
